## Safe Excel loader
This cell attempts to load `Online Retail.xlsx` from multiple locations (local and `/mnt/data`) and performs initial cleaning. Run this cell first.

In [ ]:
# ---------------------------------------------------------------------
# SAFE LOADER for "Online Retail.xlsx"
# Works on both local (C:/Users/Athil/Desktop/...) and /mnt/data paths
# ---------------------------------------------------------------------

import os
import pandas as pd
import numpy as np
from pathlib import Path

# Candidate paths to try
candidate_paths = [
    r"C:\Users\Athil\Desktop\Online Retail.xlsx",   # your main path
    r"C:\Users\Athil\Downloads\Online Retail.xlsx",
    r"C:\Users\Athil\Documents\Online Retail.xlsx",
    "/mnt/data/Online Retail.xlsx",                 # fallback (this chat environment)
    "Online Retail.xlsx"                            # current working dir
]

df = None  # ensure df is defined
file_path = None
last_error = None

# Try reading file
for path in candidate_paths:
    try:
        if Path(path).exists():
            print(f"📂 Found file: {path}")
            df = pd.read_excel(path, engine="openpyxl")
            file_path = path
            print(f"✅ Loaded Excel successfully from: {path}")
            break
        else:
            print(f"❌ Not found: {path}")
    except Exception as e:
        last_error = e
        print(f"⚠️ Failed to read {path}: {e}")

# If not loaded, stop execution
if df is None:
    raise FileNotFoundError(
        f\"\"\"❌ Could not find or read 'Online Retail.xlsx' in any known location.
Last error: {last_error}

Please verify the file path and try again.\"\"\"
    )

# ---------------------------------------------------------------------
# Basic Cleaning
# ---------------------------------------------------------------------
print("\\n🧹 Cleaning data...")

df.columns = [c.strip().replace(" ", "_") for c in df.columns]

if 'InvoiceDate' not in df.columns:
    raise KeyError("Expected column 'InvoiceDate' not found in Excel file!")

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Drop missing or invalid data
df = df.dropna(subset=['CustomerID'])
df['CustomerID'] = df['CustomerID'].astype(int)

# Remove cancellations (InvoiceNo with 'C' or negative Quantity)
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df = df[~df['InvoiceNo'].str.contains('C', na=False)]
df = df[df['Quantity'] > 0]

# Compute TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print(f"✅ Cleaned dataset: {len(df)} rows, {df['CustomerID'].nunique()} unique customers.")
print(f"📅 Date range: {df['InvoiceDate'].min().date()} → {df['InvoiceDate'].max().date()}")
print(f"💰 Total revenue: {df['TotalPrice'].sum():,.2f}")

# Optional: Save cleaned data
df.to_csv('data_cleaned.csv', index=False)
print('\\n💾 Cleaned dataset saved as data_cleaned.csv (relative path)')

# 🧠 RFM & Cohort Analysis — VS Code Notebook (Local Excel Path)

This version is configured to load your dataset directly from your **local machine path**:
```
C:/Users/Athil/Desktop/Online Retail.xlsx
```
Make sure the Excel file exists in that location before running.

---

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
pd.options.display.max_columns = 100
print('✅ Libraries imported successfully!')

## 2. Load Local Excel File

In [ ]:
file_path = 'C:/Users/Athil/Desktop/Online Retail.xlsx'

# Load the Excel dataset from your system
try:
    df = pd.read_excel(file_path)
    print('✅ Data loaded successfully from:', file_path)
except FileNotFoundError:
    print('❌ File not found! Please check your file path:', file_path)

df.head()

## 3. Data Preprocessing

In [ ]:
# Standardize column names
df.columns = [c.strip().replace(' ', '_') for c in df.columns]

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Drop missing Customer IDs and cancelled invoices
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.contains('C', na=False)]

# Compute TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
print('✅ Data cleaned successfully!')
df.head()

## 4. RFM Calculation

In [ ]:
# Snapshot date for recency calculation
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
rfm = rfm[rfm['Monetary'] > 0]
print('✅ RFM table created!')
rfm.head()

## 5. Assign RFM Segments

In [ ]:
rfm['R_score'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1]).astype(int)
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
rfm['M_score'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4]).astype(int)

rfm['RFM_Sum'] = rfm['R_score'] + rfm['F_score'] + rfm['M_score']

def rfm_segment(row):
    if row['RFM_Sum'] >= 10:
        return 'Champions'
    elif row['RFM_Sum'] >= 8:
        return 'Loyal'
    elif row['RFM_Sum'] >= 6:
        return 'Potential'
    elif row['RFM_Sum'] >= 4:
        return 'Needs Attention'
    else:
        return 'At Risk'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)
rfm.head()

### Visualize Segment Distribution

In [ ]:
plt.figure(figsize=(8,5))
rfm['Segment'].value_counts().plot(kind='bar')
plt.title('Customer Segments by RFM Score')
plt.xlabel('Segment')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Cohort Analysis (Customer Retention)

In [ ]:
# Define cohort month and retention
cohort_data = df.copy()
cohort_data['InvoiceMonth'] = cohort_data['InvoiceDate'].dt.to_period('M').dt.to_timestamp()
first_purchase = cohort_data.groupby('CustomerID')['InvoiceDate'].min().dt.to_period('M').dt.to_timestamp()
cohort_data = cohort_data.join(first_purchase, on='CustomerID', rsuffix='_Cohort')

cohort_data['CohortIndex'] = ((cohort_data['InvoiceMonth'].dt.year - cohort_data['InvoiceDate_Cohort'].dt.year) * 12 + 
                               (cohort_data['InvoiceMonth'].dt.month - cohort_data['InvoiceDate_Cohort'].dt.month) + 1)

cohort_counts = cohort_data.groupby(['InvoiceDate_Cohort', 'CohortIndex'])['CustomerID'].nunique().reset_index()
cohort_pivot = cohort_counts.pivot(index='InvoiceDate_Cohort', columns='CohortIndex', values='CustomerID')

cohort_size = cohort_pivot.iloc[:, 0]
retention = cohort_pivot.divide(cohort_size, axis=0)

plt.figure(figsize=(10,6))
plt.imshow(retention, aspect='auto', cmap='Blues')
plt.title('Cohort Retention Heatmap')
plt.xlabel('Cohort Index (Months)')
plt.ylabel('Cohort Start Month')
plt.colorbar(label='Retention Rate')
plt.tight_layout()
plt.show()

## 7. Summary Insights
- **Champions:** Highest-value, recent buyers. Maintain loyalty via personalized offers.
- **At Risk:** Long inactive customers — re-engage with discounts.
- **Retention:** Cohort analysis shows retention declines after 2–3 months; focus on onboarding.
- **Top Products:** Analyze top-selling SKUs to guide promotions.

---
**Notebook created for local use in VS Code.**